In [50]:
import os
import json
import yaml
import boto3
import logging
from globals import *
from pathlib import Path
from botocore.exceptions import ClientError

In [51]:
# Setup logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [52]:
!pygmentize globals.py

# global constants
CONFIG_FILE: str = "config.yml"
MODEL_ID_TOPROMPT_ID_MAPPING_FILE: str = ".model_id_to_prompt_id_mapping.json"

LAMBDA_DIR: str = "lambda"
CONFIG_FILE: str = "config.yml"
CODE_GEN_LAMBDA: str = "code-gen"
LAMBDA_ARN_FILE: str = ".lambda_arn"


In [53]:
config = yaml.safe_load(Path(CONFIG_FILE).read_text())
logger.info(f"config=\n{json.dumps(config, indent=2)}")

[2025-01-19 03:03:49,632] p21089 {3811199622.py:2} INFO - config=
{
  "general": {
    "app_name": "code-gen-agent",
    "description": "Amazon Bedrock Agent for generating code for the USACO benchmark",
    "role_name": "CodeGenLambdaRole",
    "region": "us-east-1",
    "model_id": "amazon.nova-micro-v1:0",
    "agent_instructions": "Generate Python code for the USACO problems. You have access to a tool for generating the code.\n",
    "ttl": 1800
  },
  "action_group": {
    "name": "Generate_Python_code",
    "description": "Generates Python code by using foundation models"
  },
  "prompt_info": {
    "name": "USACO_{model_id}",
    "description": "Generate code for the USACO benchmark"
  },
  "prompt_templates": {
    "nova": {
      "models": [
        "amazon.nova-pro-v1:0",
        "amazon.nova-lite-v1:0",
        "amazon.nova-micro-v1:0"
      ],
      "text": "Please reply with a Python 3 solution to the below problem. Read the general instructions below that are applicable t

In [54]:
!source .venv/bin/activate &&  cd {LAMBDA_DIR} && ./build_and_push.sh code_gen && cd -

export REGION=us-east-1
region=us-east-1, image=latest, account=015469603702, image=latest, repo=code_gen#, base image=
"code_gen"
code_gen already exists, no need to create it
user=ubuntu, building the container image 015469603702.dkr.ecr.us-east-1.amazonaws.com/code_gen:latest
DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon   21.5kB
Step 1/8 : FROM amazon/aws-lambda-python:3.11
 ---> c14937f0dcb9
Step 2/8 : COPY requirements.txt ${LAMBDA_TASK_ROOT}
 ---> Using cache
 ---> bed1b585a98d
Step 3/8 : RUN pip install --upgrade pip
 ---> Using cache
 ---> 74ca82e57309
Step 4/8 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> 4d63a7329e6a
Step 5/8 : RUN pip install -U boto3 botocore
 ---> Using cache
 ---> d4a4acc5348a
Step 6/8 : COPY index.py ${LAMBDA_TASK_ROOT}
 ---> cb2ff281e94e
Ste

In [55]:
# create a lambda function using this image
def function_exists(function_name: str, region_name: str) -> bool:
        """Check if Lambda function exists."""
        try:
            client = boto3.client("lambda", region_name=region_name)
            client.get_function(FunctionName=function_name)
            return True
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                return False
            raise

IMAGE_URI_FILE: str = os.path.join(LAMBDA_DIR, "image")
image = Path(IMAGE_URI_FILE).read_text().strip()
role_name: str = config['general']['role_name']
account: str = boto3.client('sts').get_caller_identity()['Account']
role = f"arn:aws:iam::{account}:role/{role_name}"
memory = 1024*10
timeout: int = 90
region_name: str = config['general']['region']
lambda_name: str = CODE_GEN_LAMBDA

client = boto3.client("lambda", region_name=region_name)

if function_exists(lambda_name, region_name) is not True:
    logger.info(f"{lambda_name} lambda does not exist, creating it now")
    logger.info(f"building lambda function with lambda_name={lambda_name}, image={image}, region_name={region_name}, role={role}, memory={memory}, timeout={timeout}")

    response = client.create_function(
        PackageType='Image',  # Specify we're using a container image
        Code={
            'ImageUri': image
        },
        Description='Generate code using Amazon Bedrock',
        Environment={
            'Variables': {
                'MODEL_ID_TO_PROMPT_ID_MAPPING': Path(MODEL_ID_TOPROMPT_ID_MAPPING_FILE).read_text()
            },
        },
        FunctionName=lambda_name,
        MemorySize=memory,
        Publish=True,
        Role=role,
        Tags={
            'TEAM': 'GenAIDevs',
        },
        Timeout=timeout,
        TracingConfig={
            'Mode': 'Active',
        },
        Architectures=['x86_64']  # or ['arm64'] for ARM-based
    )
    logger.info(response)
else:
     logger.info(f"{lambda_name} lambda does exist, updating it with latest code")
     response = client.update_function_code(
                    FunctionName=lambda_name,
                    ImageUri=image,
                    Publish=True,
                )
     logger.info(response)
     

[2025-01-19 03:03:53,718] p21089 {2528287115.py:55} INFO - code-gen lambda does exist, updating it with latest code
[2025-01-19 03:03:54,551] p21089 {2528287115.py:61} INFO - {'ResponseMetadata': {'RequestId': 'a6cd5f16-ceb8-48e0-aa5c-3dc30431e1cf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 19 Jan 2025 03:03:54 GMT', 'content-type': 'application/json', 'content-length': '1532', 'connection': 'keep-alive', 'x-amzn-requestid': 'a6cd5f16-ceb8-48e0-aa5c-3dc30431e1cf'}, 'RetryAttempts': 0}, 'FunctionName': 'code-gen', 'FunctionArn': 'arn:aws:lambda:us-east-1:015469603702:function:code-gen:16', 'Role': 'arn:aws:iam::015469603702:role/CodeGenLambdaRole', 'CodeSize': 0, 'Description': 'Generate code using Amazon Bedrock', 'Timeout': 90, 'MemorySize': 10240, 'LastModified': '2025-01-19T03:03:54.000+0000', 'CodeSha256': 'ae4e9b8ddaf9ce9b4bf4259125fed6ac41d348db737129450409f27b15044c95', 'Version': '16', 'Environment': {'Variables': {'MODEL_ID_TO_PROMPT_ID_MAPPING': '{\n  "amazon.nova-

In [56]:
lambda_arn: str = response['FunctionArn']
logger.info(f"lambda creation or update done, arn={lambda_arn}")
Path(LAMBDA_ARN_FILE).write_text(lambda_arn)

[2025-01-19 03:03:54,559] p21089 {4097021626.py:2} INFO - lambda creation or update done, arn=arn:aws:lambda:us-east-1:015469603702:function:code-gen:16


58